In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

# Load the dataset
file_path = "/kaggle/input/rt-iot2022/RT_IOT2022.csv"  # Update with correct path
df = pd.read_csv(file_path)

# Check class distribution
print(df['Attack_type'].value_counts(normalize=True) * 100)


In [ ]:
!pip install torch-geometric torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-2.0.0+cpu.html


In [ ]:
import torch
import pandas as pd
import networkx as nx
from torch_geometric.data import Data
from sklearn.preprocessing import LabelEncoder

# Load the dataset
file_path = "/kaggle/input/rt-iot2022/RT_IOT2022.csv"  # Update path if needed
df = pd.read_csv(file_path)

# Step 1: Create a mapping of ports to unique node indices
unique_ports = pd.concat([df["id.orig_p"], df["id.resp_p"]]).unique()
port_to_index = {port: idx for idx, port in enumerate(unique_ports)}

# Step 2: Create edge list (source -> destination) using port indices
edge_index = torch.tensor([
    [port_to_index[src], port_to_index[dst]] 
    for src, dst in zip(df["id.orig_p"], df["id.resp_p"])
], dtype=torch.long).t()  # Transpose to match PyTorch Geometric format

# Step 3: Extract edge features (flow duration, packet counts)
edge_features = torch.tensor(df[["flow_duration", "fwd_pkts_tot", "bwd_pkts_tot"]].values, dtype=torch.float)

# Step 4: Create node features (aggregate statistics per node)
node_features_df = df.groupby("id.orig_p")[["fwd_pkts_tot", "bwd_pkts_tot", "flow_duration"]].mean().reset_index()
node_features = torch.tensor(node_features_df[["fwd_pkts_tot", "bwd_pkts_tot", "flow_duration"]].values, dtype=torch.float)

# Step 5: Encode labels (Anomaly Detection: Attack_type)
label_encoder = LabelEncoder()
labels = torch.tensor(label_encoder.fit_transform(df["Attack_type"]), dtype=torch.long)

# Create PyTorch Geometric Data object
graph_data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_features, y=labels)

# Print graph data summary
print(graph_data)


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from torch_geometric.data import DataLoader

# Define the Graph Attention Network (GAT) Model
class GATAnomalyDetector(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=2, dropout=0.3):
        super(GATAnomalyDetector, self).__init__()
        
        # First GAT Layer
        self.conv1 = GATConv(in_channels, hidden_channels, heads=heads, dropout=dropout)
        
        # Second GAT Layer
        self.conv2 = GATConv(hidden_channels * heads, out_channels, heads=1, concat=False, dropout=dropout)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)  # Activation Function
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)  # Output probabilities

# Define Model Parameters
in_channels = graph_data.x.shape[1]  # Number of node features
hidden_channels = 16
out_channels = len(torch.unique(graph_data.y))  # Number of classes (anomalies)

# Initialize Model
gat_model = GATAnomalyDetector(in_channels, hidden_channels, out_channels)
print(gat_model)


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from torch_geometric.data import Data
from torch.optim import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load dataset
df = pd.read_csv("/kaggle/input/rt-iot2022/RT_IOT2022.csv")

# Encode categorical features
categorical_columns = df.select_dtypes(include=['object']).columns
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Separate features and target variable
X = df.drop(columns=['Attack_type'])
y = df['Attack_type']

# Normalize numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Handle class imbalance using SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, stratify=y_resampled, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float)
X_test_tensor = torch.tensor(X_test, dtype=torch.float)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Convert to Graph Data
edge_index = torch.randint(0, X_train_tensor.shape[0], (2, X_train_tensor.shape[0] * 2))
graph_data = Data(x=X_train_tensor, edge_index=edge_index, y=y_train_tensor)

def preprocess_graph_data(graph_data):
    """Ensure valid edge indices."""
    num_nodes = graph_data.x.size(0)
    valid_edges_mask = (graph_data.edge_index[0] < num_nodes) & (graph_data.edge_index[1] < num_nodes)
    graph_data.edge_index = graph_data.edge_index[:, valid_edges_mask]
    return graph_data

graph_data = preprocess_graph_data(graph_data)

# Define GAT Model
class GATAnomalyDetector(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=2, dropout=0.3):
        super(GATAnomalyDetector, self).__init__()
        self.conv1 = GATConv(in_channels, hidden_channels, heads=heads, dropout=dropout)
        self.conv2 = GATConv(hidden_channels * heads, out_channels, heads=1, concat=False, dropout=dropout)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Initialize Model
in_channels = X_train_tensor.shape[1]
hidden_channels = 16
out_channels = len(torch.unique(y_train_tensor))
gat_model = GATAnomalyDetector(in_channels, hidden_channels, out_channels)

# Optimizer and Loss
optimizer = Adam(gat_model.parameters(), lr=0.005, weight_decay=5e-4)
loss_fn = torch.nn.CrossEntropyLoss()

# Training Function
def train():
    gat_model.train()
    optimizer.zero_grad()
    out = gat_model(graph_data.x, graph_data.edge_index)
    loss = loss_fn(out, graph_data.y)
    loss.backward()
    optimizer.step()
    return loss.item()

# Evaluate Model
def evaluate_model(model, data, show_plots=True):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        pred = out.argmax(dim=1)
        y_true = data.y.cpu().numpy()
        y_pred = pred.cpu().numpy()

        # Compute metrics
        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred, average='weighted', zero_division=1)
        recall = recall_score(y_true, y_pred, average='weighted', zero_division=1)
        f1 = f1_score(y_true, y_pred, average='weighted')

        print("\nClassification Report:\n", classification_report(y_true, y_pred))
        print(f"\nAccuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")

        # Plot Confusion Matrix
        if show_plots:
            cm = confusion_matrix(y_true, y_pred)
            plt.figure(figsize=(8, 6))
            sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=np.unique(y_true), yticklabels=np.unique(y_true))
            plt.xlabel("Predicted Label")
            plt.ylabel("True Label")
            plt.title("Confusion Matrix")
            plt.show()

        return {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'predictions': y_pred,
            'true_labels': y_true
        }

# Train Model
num_epochs = 2000
for epoch in range(num_epochs):
    loss = train()
    if (epoch + 1) % 5 == 0:
        eval_metrics = evaluate_model(gat_model, graph_data, show_plots=False)
        print(f"Epoch {epoch+1}: Loss={loss:.4f}, Accuracy={eval_metrics['accuracy']:.4f}, F1-Score={eval_metrics['f1_score']:.4f}")

# Final Evaluation with Confusion Matrix
final_metrics = evaluate_model(gat_model, graph_data, show_plots=True)


In [ ]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-2.0.0+cpu.html

In [ ]:
torch.save(gat_model.state_dict(), "gat_teacher.pth")
print("Teacher model saved successfully! ✅")


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from torch.optim import Adam
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch_geometric.data import Data
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.over_sampling import SMOTE

# Load dataset
df = pd.read_csv("/kaggle/input/rt-iot2022/RT_IOT2022.csv")

# Encode categorical features
categorical_columns = df.select_dtypes(include=['object']).columns
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Features & Labels
X = df.drop(columns=['Attack_type'])
y = df['Attack_type']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Handle class imbalance using SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, stratify=y_resampled, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float)
X_test_tensor = torch.tensor(X_test, dtype=torch.float)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Create Graph Data
edge_index = torch.randint(0, X_train_tensor.shape[0], (2, X_train_tensor.shape[0] * 2))
graph_data = Data(x=X_train_tensor, edge_index=edge_index, y=y_train_tensor)

# Define Teacher Model
class GATTeacher(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=2, dropout=0.3):
        super(GATTeacher, self).__init__()
        self.conv1 = GATConv(in_channels, hidden_channels, heads=heads, dropout=dropout)
        self.conv2 = GATConv(hidden_channels * heads, out_channels, heads=1, concat=False, dropout=dropout)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Initialize Teacher Model
in_channels = X_train_tensor.shape[1]
hidden_channels = 16
out_channels = len(torch.unique(y_train_tensor))
gat_teacher = GATTeacher(in_channels, hidden_channels, out_channels)

# Optimizer & Loss
optimizer = Adam(gat_teacher.parameters(), lr=0.005, weight_decay=5e-4)
loss_fn = torch.nn.CrossEntropyLoss()

# Train Teacher Model
def train_teacher():
    gat_teacher.train()
    optimizer.zero_grad()
    out = gat_teacher(graph_data.x, graph_data.edge_index)
    loss = loss_fn(out, graph_data.y)
    loss.backward()
    optimizer.step()
    return loss.item()

# Training loop
num_epochs_teacher = 500
for epoch in range(num_epochs_teacher):
    loss = train_teacher()
    if (epoch + 1) % 5 == 0:
        print(f"Teacher Epoch {epoch+1}: Loss={loss:.4f}")

# Save Teacher Model
torch.save(gat_teacher.state_dict(), "gat_teacher.pth")
print("✅ Teacher Model Training Complete & Saved")


In [ ]:
# Load Teacher Model
gat_teacher = GATTeacher(in_channels, hidden_channels, out_channels)
gat_teacher.load_state_dict(torch.load("gat_teacher.pth"))
gat_teacher.eval()

# Define Student Model
class GATStudent(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=1, dropout=0.3):
        super(GATStudent, self).__init__()
        self.conv1 = GATConv(in_channels, hidden_channels, heads=heads, dropout=dropout)
        self.conv2 = GATConv(hidden_channels * heads, out_channels, heads=1, concat=False, dropout=dropout)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Initialize Student Model
hidden_channels_student = 8  # Smaller hidden layer
gat_student = GATStudent(in_channels, hidden_channels_student, out_channels)

# Optimizer for Student
student_optimizer = Adam(gat_student.parameters(), lr=0.005, weight_decay=5e-4)

# Distillation Loss Function
def distillation_loss(student_logits, teacher_logits, labels, temperature=3, alpha=0.7):
    soft_targets = F.softmax(teacher_logits / temperature, dim=1)
    student_probs = F.log_softmax(student_logits / temperature, dim=1)
    kl_div_loss = F.kl_div(student_probs, soft_targets, reduction="batchmean") * (temperature ** 2)
    ce_loss = F.cross_entropy(student_logits, labels)
    return alpha * kl_div_loss + (1 - alpha) * ce_loss

# Train Student Model
def train_student():
    gat_student.train()
    student_optimizer.zero_grad()

    with torch.no_grad():
        teacher_outputs = gat_teacher(graph_data.x, graph_data.edge_index)

    student_outputs = gat_student(graph_data.x, graph_data.edge_index)

    loss = distillation_loss(student_outputs, teacher_outputs, graph_data.y)

    loss.backward()
    student_optimizer.step()
    
    return loss.item()

# Training loop
num_epochs_student = 500
for epoch in range(num_epochs_student):
    loss = train_student()
    if (epoch + 1) % 5 == 0:
        print(f"Student Epoch {epoch+1}: Loss={loss:.4f}")

# Save Student Model
torch.save(gat_student.state_dict(), "gat_student.pth")
print("✅ Student Model Training Complete & Saved")


# -------------------------------
# 📌 Student Model Evaluation
# -------------------------------

# Filter edge_index to ensure all indices are within valid range
valid_mask = (edge_index[0] < X_test.shape[0]) & (edge_index[1] < X_test.shape[0])
edge_index = edge_index[:, valid_mask]

# Verify the fix
max_index_after = edge_index.max().item()
print(f"Max index in edge_index after filtering: {max_index_after}, Node count: {X_test.shape[0]}")

def evaluate_student():
    gat_student.eval()
    with torch.no_grad():
        student_outputs = gat_student(X_test_tensor, edge_index)
        student_predictions = student_outputs.argmax(dim=1).cpu().numpy()

    y_test_np = y_test_tensor.cpu().numpy()

    accuracy = accuracy_score(y_test_np, student_predictions)
    precision = precision_score(y_test_np, student_predictions, average='weighted', zero_division=1)
    recall = recall_score(y_test_np, student_predictions, average='weighted', zero_division=1)
    f1 = f1_score(y_test_np, student_predictions, average='weighted', zero_division=1)

    print("\n🎯 **Student Model Evaluation Results**:")
    print(f"✅ Accuracy: {accuracy:.4f}")
    print(f"✅ Precision: {precision:.4f}")
    print(f"✅ Recall: {recall:.4f}")
    print(f"✅ F1 Score: {f1:.4f}")

# Run evaluation
evaluate_student()


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

def evaluate_student():
    gat_student.eval()
    with torch.no_grad():
        student_outputs = gat_student(X_test_tensor, edge_index)
        student_predictions = student_outputs.argmax(dim=1).cpu().numpy()

    y_test_np = y_test_tensor.cpu().numpy()

    accuracy = accuracy_score(y_test_np, student_predictions)
    precision = precision_score(y_test_np, student_predictions, average='weighted', zero_division=1)
    recall = recall_score(y_test_np, student_predictions, average='weighted', zero_division=1)
    f1 = f1_score(y_test_np, student_predictions, average='weighted', zero_division=1)

    print("\n🎯 **Student Model Evaluation Results**:")
    print(f"✅ Accuracy: {accuracy:.4f}")
    print(f"✅ Precision: {precision:.4f}")
    print(f"✅ Recall: {recall:.4f}")
    print(f"✅ F1 Score: {f1:.4f}")

    # Create confusion matrix
    cm = confusion_matrix(y_test_np, student_predictions)

    # Plot confusion matrix
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title("Confusion Matrix")
    plt.show()

# Run evaluation
evaluate_student()


In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Load the trained student model
student_model = tf.keras.models.load_model("student_model_fixed.h5")

# Make predictions
student_preds = student_model.predict(X_test)
student_preds_binary = (student_preds > 0.5).astype(int)

# Compute evaluation metrics
accuracy_student = accuracy_score(y_test, student_preds_binary)
roc_auc_student = roc_auc_score(y_test, student_preds_binary)
conf_matrix = confusion_matrix(y_test, student_preds_binary)

# Print results
print("\n🔹 Student Model Accuracy:", accuracy_student)
print("\n🔹 Student Model ROC-AUC Score:", roc_auc_student)
print("\n🔹 Classification Report (Student Model):\n", classification_report(y_test, student_preds_binary))

# Plot Confusion Matrix
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["Normal", "Attack"], yticklabels=["Normal", "Attack"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix - Student Model")
plt.show()
